# RECOMENDADOR DE PELICULAS

## DATASET: https://www.kaggle.com/datasets/parasharmanas/movie-recommendation-system?select=ratings.csv

## PRIMERO SE  CREARA UNO ESPECIFICO

## POSTERIORMENTE SE CREARA UN ALGORITMO DONDE SE CREAN PARA TODOS LOS USUARIOS, SEGUN LOS 3 GENEROS QUE MAS LE GUSTAN Y MAS RECIENTES LAS PELICULAS.
#### ESTE ALGORITMO SE CREARA DE FORMA INDEPENDIENTE, VER ARCHIVO ALGORITMO_RECOMENDADOR



## CARGA DE LAS LIBRERIAS

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np

import math

import matplotlib.pyplot as plt

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 1. CARGA DE LOS DATOS  DE PELICULAS Y HACEMOS UN ENCODER DE LOS GENEROS Y AÑADIMOS EL AÑO Y UN CONTADOR

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/GRADO E IA Y BIGDATA/Colab Notebooks/4-GITHUB/7-RECOMENDADOR PELICULAS/movies.csv')
print(data.shape)
for i in range(len(data)):
    particion = data.loc[i,"genres"].split(sep="|")
    for j in range(len(particion)):
        variable = particion[j]
        # print(variable)
        if variable not in data.columns:
            data[variable]=0
            data.loc[i,variable]=1
        else:
            data.loc[i,variable]=1
data['year'] = data['title'].str.extract(r'\((\d{4})\)$')
print(data.columns)
print(data['year'].isna().sum())
data = data.dropna()
data['year']= data['year'].astype(int)
data['contador'] = 1
print(data.shape)


(62423, 3)
Index(['movieId', 'title', 'genres', 'Adventure', 'Animation', 'Children',
       'Comedy', 'Fantasy', 'Romance', 'Drama', 'Action', 'Crime', 'Thriller',
       'Horror', 'Mystery', 'Sci-Fi', 'IMAX', 'Documentary', 'War', 'Musical',
       'Western', 'Film-Noir', '(no genres listed)', 'year'],
      dtype='object')
566
(61857, 25)


#### REPASAMOS LOS CAMPOS EN BUSCA DE ERRORES

In [ ]:
for columna in data.columns:
    if data[columna].dtype == 'object':
        print(columna)
        print(data[columna].unique())
    else:
        print(columna)
        print(data[columna].unique())



movieId
[     1      2      3 ... 209163 209169 209171]
title
['Toy Story (1995)' 'Jumanji (1995)' 'Grumpier Old Men (1995)' ...
 'Bad Poems (2018)' 'A Girl Thing (2001)' "Women of Devil's Island (1962)"]
genres
['Adventure|Animation|Children|Comedy|Fantasy'
 'Adventure|Children|Fantasy' 'Comedy|Romance' ...
 'Children|Comedy|Fantasy|Mystery' 'Children|Comedy|Crime|Fantasy'
 'Comedy|Horror|Mystery|Sci-Fi|Western']
Adventure
[1 0]
Animation
[1 0]
Children
[1 0]
Comedy
[1 0]
Fantasy
[1 0]
Romance
[0 1]
Drama
[0 1]
Action
[0 1]
Crime
[0 1]
Thriller
[0 1]
Horror
[0 1]
Mystery
[0 1]
Sci-Fi
[0 1]
IMAX
[0 1]
Documentary
[0 1]
War
[0 1]
Musical
[0 1]
Western
[0 1]
Film-Noir
[0 1]
(no genres listed)
[0 1]
year
[1995 1994 1996 1976 1992 1988 1967 1993 1964 1977 1965 1982 1990 1991
 1989 1937 1940 1969 1981 1973 1970 1960 1955 1959 1968 1980 1975 1986
 1948 1943 1950 1946 1987 1997 1974 1956 1958 1949 1972 1998 1933 1952
 1951 1957 1961 1954 1934 1944 1963 1942 1941 1953 1939 1947 1945 1938
 1935

## CARGAMOS LOS USUARIOS

In [ ]:
ratings = pd.read_csv('/content/drive/MyDrive/GRADO E IA Y BIGDATA/Colab Notebooks/4-GITHUB/7-RECOMENDADOR PELICULAS/ratings.csv')
print(ratings.head())

   userId  movieId  rating   timestamp
0       1      296     5.0  1147880044
1       1      306     3.5  1147868817
2       1      307     5.0  1147868828
3       1      665     5.0  1147878820
4       1      899     3.5  1147868510


## SELECCIONAMOS UN USUARIO Y TRABAJAMOS SOBRE EL
#### ESTO ES PARA AHORRAR MEMORIA EN EL COLAB
#### MODIFICAR EL PROGRAMA SOLO MODIFICAR EL USUSARIO

In [ ]:
numerouserdata=4
userdata= ratings[ratings['userId']==numerouserdata]
usergustos = data[data['movieId'].isin(userdata['movieId'])]
usergustos['puntos'] = ratings[ratings['userId']==numerouserdata]['rating']
print(usergustos[usergustos['puntos']>=4].sum())

movieId                                                            2116
title                 Die Hard (1988)Monty Python's Life of Brian (1...
genres                                      Action|Crime|ThrillerComedy
Adventure                                                             0
Animation                                                             0
Children                                                              0
Comedy                                                                1
Fantasy                                                               0
Romance                                                               0
Drama                                                                 0
Action                                                                1
Crime                                                                 1
Thriller                                                              1
Horror                                                          

/tmp/ipython-input-4257078904.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usergustos['puntos'] = ratings[ratings['userId']==numerouserdata]['rating']


#### BUSCADOR DE GENERO

In [ ]:
from functools import total_ordering
puntaje = 4
generodict= {0:'Adventure', 1:'Animation', 2:'Children', 3:'Comedy', 4:'Fantasy',
             5:'Romance', 6:'Drama', 7:'Action', 8:'Crime', 9:'Thriller',
             10:'Horror', 11:'Mystery', 12:'Sci-Fi', 13:'IMAX', 14:'Documentary',
             15:'War', 16:'Musical', 17:'Western', 18:'Film-Noir'}

genero=[]
genero.append(usergustos[usergustos['puntos']>=puntaje]['Adventure'].sum())
genero.append(usergustos[usergustos['puntos']>=puntaje]['Animation'].sum())
genero.append(usergustos[usergustos['puntos']>=puntaje]['Children'].sum())
genero.append(usergustos[usergustos['puntos']>=puntaje]['Comedy'].sum())
genero.append(usergustos[usergustos['puntos']>=puntaje]['Fantasy'].sum())
genero.append(usergustos[usergustos['puntos']>=puntaje]['Romance'].sum())
genero.append(usergustos[usergustos['puntos']>=puntaje]['Drama'].sum())
genero.append(usergustos[usergustos['puntos']>=puntaje]['Action'].sum())
genero.append(usergustos[usergustos['puntos']>=puntaje]['Crime'].sum())
genero.append(usergustos[usergustos['puntos']>=puntaje]['Thriller'].sum())
genero.append(usergustos[usergustos['puntos']>=puntaje]['Horror'].sum())
genero.append(usergustos[usergustos['puntos']>=puntaje]['Mystery'].sum())
genero.append(usergustos[usergustos['puntos']>=puntaje]['Sci-Fi'].sum())
genero.append(usergustos[usergustos['puntos']>=puntaje]['IMAX'].sum())
genero.append(usergustos[usergustos['puntos']>=puntaje]['Documentary'].sum())
genero.append(usergustos[usergustos['puntos']>=puntaje]['War'].sum())
genero.append(usergustos[usergustos['puntos']>=puntaje]['Musical'].sum())
genero.append(usergustos[usergustos['puntos']>=puntaje]['Western'].sum())
genero.append(usergustos[usergustos['puntos']>=puntaje]['Film-Noir'].sum())
total = (usergustos[usergustos['puntos']>=puntaje]['contador'].sum())


abuscar=[]
for i in range(len(genero)):
    if i < 19:
        if genero[i] > (total * 0.25) :
            abuscar.append(1)
        else:
            abuscar.append(0)
print(abuscar)

totalabuscar = 0
for i in range(len(abuscar)):
    if abuscar[i] == 1:
        totalabuscar = totalabuscar + 1

print(totalabuscar)


if totalabuscar > 3:
    # Crear lista de pares (valor, índice_original)
    indexed_genero = [(genero[i], i) for i in range(len(genero))]
    # Ordenar por valor descendente
    indexed_genero.sort(reverse=True)
    # Obtener los índices originales de los 3 primeros
    top_indices = [indexed_genero[i][1] for i in range(3)]

    # Actualizar abuscar
    for i in range(len(abuscar)):
        abuscar[i] = 1 if i in top_indices else 0

print(abuscar)

peliculas= data[(data['Adventure'] == abuscar[0]) \
                & (data['Animation'] == abuscar[1])  \
                & (data['Children'] == abuscar[2]) \
                & (data['Comedy'] == abuscar[3]) \
                & (data['Fantasy'] == abuscar[4]) \
                & (data['Romance'] == abuscar[5]) \
                & (data['Drama'] == abuscar[6]) \
                & (data['Action'] == abuscar[7]) \
                & (data['Crime'] == abuscar[8]) \
                & (data['Thriller'] == abuscar[9]) \
                & (data['Horror'] == abuscar[10]) \
                & (data['Mystery'] == abuscar[11]) \
                & (data['Sci-Fi'] == abuscar[12]) \
                & (data['IMAX'] == abuscar[13]) \
                & (data['Documentary'] == abuscar[14]) \
                & (data['War'] == abuscar[15]) \
                & (data['Musical'] == abuscar[16]) \
                & (data['Western'] == abuscar[17]) \
                & (data['Film-Noir'] == abuscar[18])

               ]

# print(peliculas['contador'].sum())

# print(peliculas['title'])

# print(peliculas['year'].max())

cartelera = peliculas[peliculas['year']==peliculas['year'].max()]



print(cartelera['title'])


[0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
4
[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
59419    John Wick: Chapter 3 – Parabellum (2019)
61132                           21 Bridges (2019)
61956                            Lucky Day (2019)
Name: title, dtype: object
